### **Linear Regression Problem (LRP)**

* Copyright (c) 2022-2024, Keivan Tafakkori. All rights reserved.
* See the file LICENSE file for licensing details.

####  *Required modules*

In [71]:
import feloopy as flp
import numpy as np

####  Dataset

In [72]:
dt = flp.data_toolkit(key=0)

train_a = dt.store(name="train_a", value=np.array([[1, 2, 2], [2, 3, 3], [3, 4, 5], [4, 5, 6], [ 5, 7, 8]]))
test__a = dt.store(name="test__a", value=np.array([[6, 8, 9], [7, 4, 3], [8, 3, 5], [9, 1, 6], [10, 9, 3]]))  
train_b = dt.store(name="train_b", value=np.array([1, 2, 3, 4,  5]))
test__b = dt.store(name="test__b", value=np.array([6, 7, 8, 9, 10]))
U = dt.set(name="U", bound=[0,np.shape(train_a)[1]-1])
T = dt.set(name="T", bound=[0,np.shape(train_a)[0]-1])

####  Preprocessing

In [73]:
ran_a = np.array([np.ptp(train_a[:,i])     for i in U])
ave_a = np.array([np.average(train_a[:,i]) for i in U])
nor_a = (train_a-ave_a)/ran_a
ran_b = np.ptp(train_b)
ave_b = np.average(train_b)
nor_b = (train_b-ave_b)/ran_b

####  Exact Optimization Algorithms

In [74]:
def lrp(m):
    x = m.fvar('x', [U])
    z = m.fvar('z')
    m.obj((2*len(T))**(-1)*sum((sum(nor_a[t, i]*x[i] for i in U) + z-nor_b[t])**2 for t in T))
    return m

m = flp.search(name="lrp",
               environment=lrp, 
               directions=["min"],
               method="exact",
               interface="gekko",
               solver="apopt")

m.clean_report()


√ Healthy

┌─ FelooPy v0.3.0 ───────────────────────────────────────────────── Released April 2024 ─┐
│                                                                                        │
│ Date: 2024-04-16                                                      Interface: gekko │
│ Time: 14:51:13                                                           Solver: apopt │
│ Name: lrp                                                                Method: exact │
│ Type: single-objective                                                  X Unconfigured │
│                                                                                        │
└────────────────────────────────────────────────────────────────────────────────────────┘

┌─ Model ────────────────────────────────────────────────────────────────────────────────┐
│                       B       I       P       F       E       S       O       C        │
╞════════════════════════════════════════════════════════════════════════════

####  Postprocessing

In [75]:
weight = m.get("x")
bias = m.get("z")

In [76]:
def approximator_model(weight,bias,input): 
    print(f"Input = {input*ran_a+ave_a} -> Output = {(sum(weight[i]*input[i] for i in range(len(input))) + bias)*ran_b+ave_b}")

nor_a = (test__a-ave_a)/ran_a
nor_b = (test__b-ave_b)/ran_b

for item in nor_a:
    approximator_model(weight,bias,item)

Input = [6. 8. 9.] -> Output = 5.999999623185958
Input = [7. 4. 3.] -> Output = 7.000004870167626
Input = [8. 3. 5.] -> Output = 8.000004423259778
Input = [9. 1. 6.] -> Output = 9.000004714611169
Input = [10.  9.  3.] -> Output = 10.000007364883249
